In [102]:
# importanto bibliotecas
import pandas as pd
import numpy as np
import psycopg2

#pacotes gráficos
import seaborn as sns
import matplotlib.pyplot as plt

#pacote de validação de email
from validate_email  import validate_email

# biblioteca de statística
import statistics
from sklearn.model_selection import StratifiedShuffleSplit
import scipy.stats as stats

#biblioteca de sistema operacional
import os

In [103]:
# conexão redshift 
connection = psycopg2.connect(dbname="dw_amorsaude",

                  user="lucasmateus",

                  password="df93aS@aK0sd",

                  host ="amorsaude-cluster-redshift.cmc4ev2oyczd.us-east-1.redshift.amazonaws.com",

                  port = 5439)

In [104]:
query = '''
WITH particular AS (
	SELECT
		cr.id_paciente
	FROM tb_consolidacao_contas_a_receber_hist_nova cr
	WHERE 1=1
		AND cr.datapagamento >= '2023-01-01'
		AND cr.nome_tabela_particular = 'PARTICULAR*')
SELECT 
	sp.id,
	sp.celular 
FROM stg_pacientes sp
JOIN particular pa ON sp.id = pa.id_paciente
WHERE 1=1
	AND sp.celular IS NOT NULL
	AND sp.celular <> ''
'''

In [105]:
# setando o dataframe completo

df = pd.read_sql_query(query,con=connection)

/tmp/ipykernel_4789/2586117790.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,con=connection)


In [106]:
df

,id,celular
0,57846186,(62) 99140-5502
1,60033500,(73) 99860-8548
2,60033500,(73) 99860-8548
3,60033500,(73) 99860-8548
4,60033500,(73) 99860-8548
...,...,...
1518421,64119954,(11) 94890-3400
1518422,64219769,(19) 98206-7824
1518423,64202554,(21) 96663-9324
1518424,64250881,(31) 97218-4800


In [107]:
df_particular = pd.read_excel('/home/lucasmateus/dbt-projects/projetos_lucas/amostras_psat_hubspot/base-particular-target.xlsx')


In [108]:
df_particular

,ID do Paciente,Data do Agendamento
0,67677488.0,2023-11-16
1,67703291.0,2023-11-16
2,61621777.0,2023-11-16
3,10146889.0,2023-11-16
4,67690916.0,2023-11-16
...,...,...
53755,65184379.0,2023-04-21
53756,64385050.0,2023-04-04
53757,18018050.0,2023-04-20
53758,18018050.0,2023-04-20


In [109]:
df_merged = pd.merge(df_particular, df, left_on='ID do Paciente', right_on='id', how='inner')

In [110]:
df_merged = df_merged.drop(labels=['id', 'Data do Agendamento'], axis=1)

In [111]:
df_merged = df_merged.drop_duplicates(subset='ID do Paciente')

In [112]:
df_merged

,ID do Paciente,celular
0,10146889.0,(71) 99335-4847
2,66026987.0,(22) 99712-8041
3,67672884.0,(75) 9253-0118
4,67673911.0,(11) 94885-1638
5,61254900.0,(75) 99290-4893
...,...,...
93876,63755477.0,(61) 98161-6617
93879,65184379.0,(75) 9196-6320
93883,10474218.0,(51) 9927-9359
93884,64385050.0,(19) 99620-0052


In [113]:
df_merged.to_excel('base_particular_target_celular.xlsx', sheet_name='base', index=False)